### 파이토치(PyTorch) 한국어 튜토리얼
* https://tutorials.pytorch.kr/

### 빠른 시작(Quickstart)
* https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html

#### 데이터 작업하기

In [29]:
import torch
from torch import nn

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

In [30]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [31]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [32]:
# 모델 만들기 - # 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### 모델 매개변수 최적화하기

In [34]:
# 모델을 학습 - loss function, optimizer 생성

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

### Dataset과 DataLoader
* https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

### 신경망 모델 구성하기
* https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html

* 신경망 - 계층(layer)/모듈(module)로 구성
* torch.nn 신경망을 구성하는데 필요한 모든 구성 요소를 제공
* PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 
* 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다.

In [ ]:
import os
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 학습 장치 얻기
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# class 정의 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
X = torch.rand(2, 28, 28, device=device)

# 2. 모델 예측 
#   model.forward() 바로 호출 X
#   logits - Raw Score
logits = model(X)

print(logits)           # 0 ~ 1 확률은 아님. 
print(logits.sum().item())
print("-" * 30)
# 의미: logits 점수들을 전부 더하면 1(100%)이 되도록 다듬기
# dim=1: 10개의 클래스(숫자)가 나열된 방향으로 계산하라
# 결과물은 "0일 확률 2%, 3일 확률 85%..." 같은 식
# 
# dim - 어떤 축으로 계산할지 
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

In [ ]:
# 모델 계층(Layer)
input_image = torch.rand(3,28,28)

print(input_image.size())
print(input_image)

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
print(flat_image)


In [ ]:
# nn.Linear - 입력에 선형 변환(linear transformation)을 적용

# class torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
#   y = wX + b 

layer1 = nn.Linear(in_features=28*28, out_features=20)

hidden1 = layer1(flat_image)
print(layer1.weight, layer1.bias)

print(hidden1.size())
print(hidden1.size())
print(hidden1)


In [ ]:
# nn.ReLU
#   비선형 활성화(activation) - 모델의 입력과 출력 사이의 관계(mapping)를 생성
#   비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 
#   신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


# nn.Sequential - 순서를 갖는 모듈의 컨테이너입
#   데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

print("-" * 30)
# nn.Softmax ------------------------ 
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)


In [ ]:
# 모델 매개변수 ------------------------
# 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
print("-" * 30)
